In [ ]:
import requests
def get_landmarks(destination):
    # Replace 'YOUR_API_KEY' with your actual Google Maps API key
    api_key = 'placeholder_api_key'
    
    # Make a request to the Places API to get popular landmarks
    url = f'https://maps.googleapis.com/maps/api/place/textsearch/json?query={destination}+landmarks&key={api_key}'
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()
            
            # Extract the landmark names and their corresponding locations
            landmarks = []
            for result in data['results']:
                name = result['name']
                location = result['geometry']['location']
                landmarks.append((name, location))
            
            # Display the landmarks with Google Map view
            for landmark in landmarks:
                name, location = landmark
                print(f'Landmark: {name}')
                print(f'Location: {location}')
                print(f'Google Map View: https://www.google.com/maps/search/?api=1&query={location["lat"]},{location["lng"]}\n')
            
            return landmarks, data
        else:
            print(f'Request to Places API failed with status code: {response.status_code}')
            # Handle the error here
            return None

In [ ]:
# Example usage
landmarks, data = get_landmarks('711 carriage hill, iowa city')

In [ ]:
if data is not None and 'error_message' in data:
    error_message = data['error_message']
    
    print(f'Error message: {error_message}')

In [ ]:
import requests



def get_lati_longi(api_key, address):

   

    url = 'https://maps.googleapis.com/maps/api/geocode/json'

   

    params = {

        "address": address,

        "key": api_key

    }



    response = requests.get(url, params=params)



    if response.status_code == 200:

        data = response.json()

        if data["status"] == "OK":

            location = data["results"][0]["geometry"]["location"]

            lat = location["lat"]

            lng = location["lng"]

            return lat, lng

        else:

            print(f"Error: {data['error_message']}")

            return 0, 0

    else:

        print("Failed to make the request.")

        return 0, 0



api_key = "placeholder_api_key"

address = '711 carriage hill, iowa city'



lati, longi = get_lati_longi(api_key, address)

print(f"Latitude: {lati}")

print(f"Longitude: {longi}")

In [ ]:
import gmaps
import gmaps.datasets
import pandas as pd


# Use google maps api
gmaps.configure(api_key='placeholder_api_key')  # Fill in with your API key
# Get the dataset
earthquake_df = gmaps.datasets.load_dataset_as_df('earthquakes')
# Get the locations from the data set
locations = earthquake_df[['latitude', 'longitude']]
# Get the magnitude from the data
weights = earthquake_df['magnitude']
# Set up your map
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig



In [2]:
# import openai
from elevenlabs import generate, play
from elevenlabs import set_api_key
import gmaps
import requests
import os
import re
import PIL.Image
from io import BytesIO
from IPython.display import Image
from openai import OpenAI


In [3]:
class VirtualCityExplorationModule:
    def __init__(self, cityName, openai_key, gmaps_key, NumberofAttractions=5) -> None:
        # self.openai_key = openai_key
        self.gmaps_key = gmaps_key
        self.client = OpenAI(api_key = openai_key)
        # self.client = OpenAI()
        gmaps.configure(api_key= gmaps_key)
        self.city_name= cityName
        self.numAttractions = NumberofAttractions

    def generateImagePrompt(self, attractionName):
        systemContent = 'You write image generation prompts for DALL-E 3 from the given input request. For example, if you were asked to write a prompt for an image about garden of God in Victor, Colorado, USA, you may output: Mountainous terrain with dense forests and trees, peaceful and serene, in the vicinity of garden of God of Victor, Colorado, USA. Shot on a Canon EOS R6 with a Canon RF 24-105mm f/4L IS USM Lens,, 4K film still, natural lighting, vibrant colors, crisp details, and soft shadows.'
        userPrompt = f'I want you to write a DALL-E image generation prompt to generate an image related to {attractionName} in the town of  {self.city_name}'
        # messages = [{'role': 'system', 'content': systemContent}, {'role': 'user', 'content': userPrompt}]
        chatgpt_text_messages = [ {"role": "system", "content": systemContent}, {'role': 'user', 'content': userPrompt} ]

        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=chatgpt_text_messages,
            temperature=1,
            max_tokens=300,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
        message = response.choices[0].message.content
        # print(message)
        return message
    
    def getCityCordinates (self):
        base_url = "https://maps.googleapis.com/maps/api/geocode/json"
        params = {
            "address": self.city_name,
            "key": self.gmaps_key
        }
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            data = response.json()
            if len(data['results'])>0:
                self.cityLocation = data['results'][0]['geometry']['location']
                print(f"{self.city_name} locates at {self.cityLocation['lat']}, {self.cityLocation['lng']}")
                return self.cityLocation
        return None, None
    
    def getTopAttractions(self, radius=3000, type='tourist_attraction'):
        cityCoord = self.getCityCordinates()
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        params = {
        "location": f"{cityCoord['lat']},{cityCoord['lng']}",
        "radius": radius,
        "type": type,
        "key": self.gmaps_key
        }
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            data = response.json()
            if 'results' in data:
                return data['results'][:self.numAttractions]
        return None
    
    def generateAttractionsImage(self):
        self.topLocations = self.getTopAttractions()
        self.city_map = self.VizTopLocationsInMap()
        print(f"there is {len(self.topLocations)} in {self.city_name}")
        # for attraction in self.topLocations:
        #     print(f"generating image for {attraction['name']}")
        #     imagePrompt = self.generateImagePrompt(attraction['name'])
        #         # Generate the image
        #     response = self.client.images.generate(
        #         model="dall-e-3",
        #         prompt=imagePrompt,
        #         n=1,
        #         size="1024x1024",
        #         quality="hd",
        #         style = 'vivid'
        #     )
        #     # Get the image URL from the response
        #     image_url = response.data[0].url
        #     # Download the image
        #     image_response = requests.get(image_url)

        #     # Create the directory if it doesn't exist
        #     if not os.path.exists(self.city_name.replace(' ', '-')):
        #         os.makedirs(self.city_name.replace(' ', '-'))
        #     if not os.path.exists(os.path.join(self.city_name.replace(' ', '-'), "VirtualCityExplorationImages")):
        #         os.makedirs(os.path.join(self.city_name.replace(' ', '-'), "VirtualCityExplorationImages"))
        #     with open(os.path.join(self.city_name.replace(' ', '-'), "VirtualCityExplorationImages", f"{attraction['name']}.png"), 'wb') as f:
        #         f.write(image_response.content)

    def VizTopLocationsInMap(self):
        # Create the base map
        # gmaps.configure(api_key=self.gmaps_key)
        city_map = gmaps.figure(center=(self.cityLocation['lat'], self.cityLocation['lng']), zoom_level=12)

        # Create the markers
        markers = []
        for location in self.topLocations:
            marker = gmaps.Marker(
                (location['geometry']['location']['lat'], location['geometry']['location']['lng']),
                info_box_content=f"Name: {location['name']}<br>Lat: {location['geometry']['location']['lat']}<br>Lng: {location['geometry']['location']['lng']}"
            )
            markers.append(marker)

        # Add the markers to the map
        marker_layer = gmaps.Markers(markers=markers)
        city_map.add_layer(marker_layer)

        # Export the city map as a PNG image
        # city_map(os.path.join(self.city_name.replace(' ', '-'), f"{self.city_name.replace(' ', '-')}.png"))

        return city_map


In [4]:
openai_key = 'placeholder_api_key'
gmaps_key = 'placeholder_api_key'

In [5]:
destination = "Los Angeles"
print("\n\nVirtual City Exploration.")
city_exploration_module = VirtualCityExplorationModule(destination, openai_key, gmaps_key, NumberofAttractions=5)
city_exploration_module.generateAttractionsImage()



Virtual City Exploration.
Los Angeles locates at 34.0549076, -118.242643
there is 5 in Los Angeles


In [8]:
city_map = city_exploration_module.city_map

In [6]:
cityLoc = city_exploration_module.topLocations

In [7]:
cityLoc[3]['name']

'Dodger Stadium'

In [20]:
cityLoc[0]['geometry']['location']['lat']

34.0637293